<center><img src="images/MLU-NEW-logo.png" alt="drawing" width="400" style="background-color:white; padding:1em;" /></center> <br/>

# <a name="0">Bedrock Agents and Knowledge Bases with Boto3 SDK </a>
## <a name="0">Module 2, Lab 2b: Text to SQL (Advanced) - with Knowledge Bases using Semantic Retrieval </a>

## Lab Overview

In this lab, we will generate SQL from text but unlike in Lab 2a, this time use LLMs to generate SQL from text using a SQlite database schema as the knowledge base. The agent then invokes the appropriate Lambda API to execute this generated SQL statement against a SQlite database.

##### Notebook Kernel
Please choose `conda_python3` as the kernel type of the top right corner of the notebook if that does not appear by default.

<div style="border: 4px solid coral; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px">
    <h4>This lab auto-cleans up resources to be frugal. </h4>
    You can visit this section (<a href="#10"> Clean-up Resources</a>) to change the setting if you need to experiment with prompts and settings. Please run clean-up resources after you are done with experiments. <br/>
</div>
<br/>

## LLM Used
Anthropic Claude 3 Sonnet

## Use-Case Overview
In this lab, we convert natural language text to SQL and execute it against the database to return the results. This enables non-tech and tech workers to easily collect results for their reports with simple, natural language-based query.

We use the Lambda function to only execute LLM-generated SQL statements against an in-memory SQLite database. Unlike lab 2a, where each lambda API implementation has a specific SQL query associated with it, in this lab it is up to the LLM to generate the SQL from natural language understanding, few-shot examples and a database schema as a Knowledge Base.

Without the knowledge base of the database schema and RAG, the SQL queries generated will have column names and table names that will never be the same as the database schema, therefore it will almost always never be syntactically correct to execute against a database.


It demonstrates how to:
1. Select the underlying foundation model (FM) for your agent 
2. Provide a clear and concise agent instruction 
3. Create and associate an action group with an API schema and a Lambda function 
4. Create and associate a knowledge base with the agent
5. Create, invoke, test, and deploy the agent
6. Demonstrate how to generate the SQL from natural language understanding via LLMs, few shot examples, and a database schema as a knowledge base
7. Clean up resources (Be Frugal)


We are using the Retrieval Augmented Generation (RAG) technique with Amazon Bedrock. A RAG implementation consists of two parts:

    1. A data pipeline that ingests that from documents (typically stored in Amazon S3) into a Knowledge Base i.e. a vector database such as Amazon OpenSearch Service Serverless (AOSS) so that it is available for lookup when a question is received.

    2. An application that receives a question from the user, looks up the knowledge base for relevant pieces of information (context) and then creates a prompt that includes the question and the context and provides it to an LLM for generating a response.

The data pipeline represents an undifferentiated heavy lifting and can be implemented using Amazon Bedrock Agents for Knowledge Base. We can now connect an S3 bucket to a vector database such as AOSS and have a Bedrock agent read the objects (html, pdf, text etc.), chunk them, and then convert these chunks into embeddings using Amazon Titan Embeddings model and then store these embeddings in AOSS. All of this without having to build, deploy, and manage the data pipeline.

#### Amazon Titan Embeddings Overview
Amazon Titan Embeddings is a text embeddings model that converts natural language text including single words, phrases, or even large documents, into numerical representations that can be used to power use cases such as search, personalization, and clustering based on semantic similarity. Optimized for text retrieval to enable retrieval augmented generation (RAG) use cases, Amazon Titan Embeddings, enables you to first convert your text data into numerical representations or vectors and then use those vectors to accurately search for relevant passages from a vector database, allowing you to make the most of your proprietary data in combination with other foundation models (FMs).



Once the data is available in the Bedrock knowledge base, then user questions can be answered using the following system design:

<center><img src="images/bedrock-agents-kb.png" alt="This image shows the retrieval augmented generation (RAG) system design setup with knowledge bases, S3, and AOSS. Knowledge corpus is ingested into a vector database using Amazon Bedrock Knowledge Base Agent and then RAG approach is used to work question answering. The question is converted into embeddings followed by semantic similarity search to get similar documents. With the user prompt being augmented with the RAG search response, the LLM is invoked to get the final raw response for the user." height="700" width="700" style="background-color:white; padding:1em;" /></center> <br/>


#### Lab Use-Case diagram for this lab 

<center><img src="images/lab2b-agents.png" alt="This image shows how to use an LLM to generate SQL from text via database schema description, then invoke the appropriate Lambda API which executes this LLM generated SQL statement to execute against a SQLlite database to return the SQL output to the user." height="700" width="700" style="background-color:white; padding:1em;" /></center> <br/>

This lab notebook has the following sections:

1. <a href="#1">Environment setup and configuration</a>
2. <a href="#2">Set up Bedrock for inference</a>
3. <a href="#3">Setup prefix variables for various agent resources</a>
4. <a href="#4">Create Lambda function for action group </a>
5. <a href="#5">Create knowledge base</a>
6. <a href="#6">Creating an agent</a>
7. <a href="#7">Deploy agent and create agent alias</a>
8. <a href="#8">Invoke agent</a>
9. <a href="#9">Multi-turn conversations / Session context management</a>
10. <a href="#10">Clean up resources</a>
11. <a href="#11">Challenge exercise and lab quiz</a>
    
Please work top to bottom of this notebook and don't skip sections as this could lead to error messages due to missing code.


----

You will be presented with two kinds of exercises throughout the notebook: activities and challenges. <br/>

| <img style="float: center;" src="images/activity.png" alt="Activity" width="125"/>| <img style="float: center;" src="images/challenge.png" alt="Challenge" width="125"/>|
| --- | --- |
|<p style="text-align:center;"> No coding is needed for an activity. You try to understand a concept, <br/>answer questions, or run a code cell.</p> |<p style="text-align:center;">Challenges are where you can practice your coding skills.</p>


Let's start by installing all required packages as specified in the `requirements.txt` file and importing several libraries.


## <a name="1">Environment setup and configuration</a>
(<a href="#0">Go to top</a>)

Before starting, let's import the required packages and configure the support variables:

In [1]:
%%capture
!pip3 install -r requirements.txt --quiet

In [2]:
import logging
import boto3
import random
import time
import zipfile
from io import BytesIO
import json
import uuid
import pprint
import os
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
from IPython.display import Markdown

from mlu_utils.agents_utils import *
from mlu_utils.agents_infra_utils_one_kb_setup import *
from mlu_utils.summarize_agent_trace import *
from mlu_utils.show_trace_widget import *

[2024-07-11 19:38:44,471] p364 {credentials.py:1075} INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [3]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

pp = pprint.PrettyPrinter(width=41, compact=True)

In [4]:
import ipywidgets as widgets
from IPython.display import JSON
out_2b_tabs_1 = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))
out_2b_summary = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))


In [5]:
#clean_up_trace_files("./trace_files/")

### <a name="2">2. Set up Bedrock for inference</a>
(<a href="#0">Go to top</a>)

To get started, set up Bedrock and instantiate an active `bedrock-runtime` to query LLMs. The code below leverages [LangChain's Bedrock integration](https://python.langchain.com/docs/integrations/llms/bedrock).
```
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

```
<div style="border: 4px solid coral; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px">
    <h4>A note on Bedrock API invocation</h4>

Amazon Bedrock is generally available for all AWS customers. The provided lab for this course currently invokes Bedrock's external endpoint, which has access to Anthropic and AI21 models.

For training and learning purposes, Amazon Bedrock is also available through an internal endpoint that allows accessing Amazon's Titan models. To use that instead, please follow the instructions in the <a href="https://w.amazon.com/bin/view/AmazonBedrock/Products/GetStarted/">Amazon Bedrock Get Started wiki page</a>.
</div>
</br>

In [6]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
s3_client = boto3.client('s3')
lambda_client = boto3.client('lambda')

bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
open_search_serverless_client = boto3.client('opensearchserverless')

session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-east-1', '339712993987')

In [7]:
import ipywidgets as widgets
from IPython.display import JSON
out = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))


In [8]:
# action on tab click for agent trace

def click_button(obj):
    with out:
        out.clear_output()
        logger.info(f'Expand JSON elements (if available) in {obj.new}')
        if obj.new == "Pre-Processing":
             with open("trace_files/preProcessingTrace_" + trace_filename_prefix + "_" + str(turn_number) + ".log", "r") as agent_trace_fp:
                display(JSON(agent_trace_fp.read()))
        elif obj.new == "Orchestration":
             with open("trace_files/orchestrationTrace_" + trace_filename_prefix + "_" + str(turn_number) + ".log", "r") as agent_trace_fp:
                display(JSON(agent_trace_fp.read()))
        elif obj.new == "Knowledge-Base":
             with open("trace_files/knowledgeBaseLookupOutput_" + trace_filename_prefix + "_" + str(turn_number) + ".log", "r") as agent_trace_fp:
                display(JSON(agent_trace_fp.read()))
        elif obj.new == "Post-Processing":
             with open("trace_files/postProcessingTrace_" + trace_filename_prefix + "_" + str(turn_number) + ".log", "r") as agent_trace_fp:
                display(JSON(agent_trace_fp.read()))


### <a name="3">3. Setup prefix variables for various agent resources</a>
(<a href="#0">Go to top</a>)


This is the same set of instructions for infrastructure setup as provided in Lab 2a and includes:
- Setup for prefix variables with various agent resources
- Create Lambda function for action group
- Create Knowledge Base 1 for SQL generation with Northwind Database
- Create Knowledge Base 2 for system design recommendation with AWS Well-architected framework
- Creating an agent


In [9]:
%%time
infra_response = setup_agent_infrastructure(schema_filename='sql_gen_agent_openapi_schema_with_kb.json', 
                                            kb_db_file_uri='kb_sqlgen', 
                                            lambda_code_uri='lambda_function_sql_gen.py')




[2024-07-11 19:38:45,699] p364 {agents_infra_utils_one_kb_setup.py:54} INFO - random_uuid :: 79d2bafb-9590-4bbc-b01b-44eed2496d61 prefix_infra :: l279d2ba prefix_iam :: l29590


Creating collection...


[2024-07-11 19:39:48,221] p364 {credentials.py:1075} INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole



Collection successfully created:


[2024-07-11 19:40:33,680] p364 {base.py:258} INFO - PUT https://u3y10f87t7q9rehee8hg.us-east-1.aoss.amazonaws.com:443/bedrock-knowledge-base-l279d2ba-kbdb-index [status:200 request:0.435s]
[2024-07-11 19:40:33,681] p364 {agents_infra_utils_one_kb_setup.py:524} INFO - response :: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'bedrock-knowledge-base-l279d2ba-kbdb-index'} 



Creating index:


[2024-07-11 19:43:01,245] p364 {agents_infra_utils_one_kb_setup.py:753} INFO - agent_name :: l279d2ba-agent-kb 
 agent_alias_name :: l279d2ba-workshop-alias 
 bucket_name :: l279d2ba-agent-kb-339712993987 
 schema_key :: l279d2ba-agent-kb-schema.json 
 knowledge_base_db_id :: 5VALKAU0SG 


CPU times: user 362 ms, sys: 91.5 ms, total: 453 ms
Wall time: 4min 15s


In [10]:
agent_name = infra_response["agent_name"]
agent_alias_name = infra_response["agent_alias_name"]
agent_role = infra_response["agent_role"]
bucket_name = infra_response["bucket_name"]
schema_key = infra_response["schema_key"]
knowledge_base_db_id = infra_response["knowledge_base_db_id"]
lambda_name = infra_response["lambda_name"]
lambda_function = infra_response["lambda_function"]
agent_bedrock_policy = infra_response["agent_bedrock_policy"]
agent_s3_schema_policy = infra_response["agent_s3_schema_policy"]
agent_role_name = infra_response["agent_role_name"]
lambda_role_name = infra_response["lambda_role_name"]
kb_db_collection_name = infra_response["kb_db_collection_name"]
kb_db_bedrock_policy = infra_response["kb_db_bedrock_policy"]
kb_db_aoss_policy = infra_response["kb_db_aoss_policy"]
kb_db_s3_policy = infra_response["kb_db_s3_policy"]
agent_kb_schema_policy = infra_response["agent_kb_schema_policy"]
kb_db_role_name = infra_response["kb_db_role_name"]
kb_db_opensearch_collection_response = infra_response["kb_db_opensearch_collection_response"]

#### Creating agent
Once the needed IAM role is created, we can use the Bedrock agent client to create a new agent. To do so we use the `create_agent` function. It requires an agent name, underline foundation model and instruction. You can also provide an agent description. Note that the agent created is not yet prepared. We will focus on preparing the agent and then using it to invoke actions and use other APIs

In [11]:
# Create agent
agent_instruction = """
Hello, I am SQL assistant. I can take a natural language question as input, analyze the intent and context, 
and generate a valid SQLite query using the database schema inside the Knowledge Base  
and then execute the query that answers the question based on the [Northwind] dataset.

If the Knowledge Base search function result did not contain enough information to construct a full query 
try to construct a query to the best of your ability based on the Northwind database schema.

Feel free to ask any questions along those lines!
Here are a few examples of questions I can help answer by generating and then executing a SQLite query:

- What are the total sales amounts by year?

- What are the top 5 most expensive products? 

- What is the total revenue for each employee?


"""
# anthropic.claude-3-sonnet-20240229-v1:0
# anthropic.claude-v2

response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description="Generate and Run SQL queries.",
    idleSessionTTLInSeconds=1800,
    foundationModel="anthropic.claude-v2",
    instruction=agent_instruction,
)

Looking at the created agent, we can see its status and agent id

Let's now store the agent id in a local variable to use it on the next steps

In [12]:
agent_id = response['agent']['agentId']
agent_id

'TOCGNTDIUP'

### Create agent action group
We will now create an agent action group that uses the Lambda function and API schema files created before.
The `create_agent_action_group` function provides this functionality. We will use `DRAFT` as the agent version since we haven't yet created an agent version or alias. To inform the agent about the action group functionalities, we will provide an action group description containing the functionalities of the action group.

In [13]:
bucket_name, schema_key

('l279d2ba-agent-kb-339712993987', 'l279d2ba-agent-kb-schema.json')

In [14]:
# Pause to make sure agent is created
time.sleep(30)
# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName='SqlQueryManagementActionGroup',
    apiSchema={
        's3': {
            's3BucketName': bucket_name,
            's3ObjectKey': schema_key
        }
    },
    description='Actions for executing a valid SQLite query that answers the question based on the Northwind database'
)

In [15]:
agent_action_group_response

{'ResponseMetadata': {'RequestId': 'bf510746-5dae-4b6d-9a69-56bf1c03d87a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 11 Jul 2024 19:43:31 GMT',
   'content-type': 'application/json',
   'content-length': '611',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'bf510746-5dae-4b6d-9a69-56bf1c03d87a',
   'x-amz-apigw-id': 'aw1bmHSYoAMERIQ=',
   'x-amzn-trace-id': 'Root=1-669035e3-492366b34ad9cbb135276c52'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-east-1:339712993987:function:l279d2ba-agent-kb-339712993987'},
  'actionGroupId': 'BTPOTJFLCQ',
  'actionGroupName': 'SqlQueryManagementActionGroup',
  'actionGroupState': 'ENABLED',
  'agentId': 'TOCGNTDIUP',
  'agentVersion': 'DRAFT',
  'apiSchema': {'s3': {'s3BucketName': 'l279d2ba-agent-kb-339712993987',
    's3ObjectKey': 'l279d2ba-agent-kb-schema.json'}},
  'createdAt': datetime.datetime(2024, 7, 11, 19, 43, 31, 823674, tzinfo=tzlocal()),
  'description': 'Actions

### Allowing agent to invoke action group Lambda
Before using our action group, we need to allow our agent to invoke the Lambda function associated to the action group. This is done via resource-based policy. Let's add the resource-based policy to the lambda function created:

In [16]:
# Create allow invoke permission on lambda
response = lambda_client.add_permission(
    FunctionName=lambda_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)

### Associating the agent to a knowledge base


In [17]:
agent_kb_description = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description=f'Use the information in the {knowledge_base_db_id} Knowledge Base to generate a valid SQLite Query to answer the questions based on the Northwind database',
    knowledgeBaseId=knowledge_base_db_id 
)

### Preparing agent
Let's create a DRAFT version of the agent that can be used for internal testing.

In [18]:
agent_prepare = bedrock_agent_client.prepare_agent(agentId=agent_id)
agent_prepare

{'ResponseMetadata': {'RequestId': '8b569376-fe84-4ad4-b78c-13a000587e73',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Thu, 11 Jul 2024 19:43:32 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': '8b569376-fe84-4ad4-b78c-13a000587e73',
   'x-amz-apigw-id': 'aw1bsEGVoAMEv_Q=',
   'x-amzn-trace-id': 'Root=1-669035e4-2d847cd92ba86ab829cf9935'},
  'RetryAttempts': 0},
 'agentId': 'TOCGNTDIUP',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2024, 7, 11, 19, 43, 32, 214709, tzinfo=tzlocal())}

### <a name="5">Create agent alias to deploy agent</a>
(<a href="#0">Go to top</a>)

We will now create an alias of the agent that can be used to deploy the agent.

In [19]:
# Pause to make sure agent is prepared
time.sleep(30)
agent_alias = bedrock_agent_client.create_agent_alias(
    agentId=agent_id,
    agentAliasName=agent_alias_name
)
# Pause to make sure agent alias is ready
time.sleep(30)

In [20]:
agent_alias

{'ResponseMetadata': {'RequestId': '5cd594f7-175a-4e43-8a5b-d56a9d578461',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Thu, 11 Jul 2024 19:44:02 GMT',
   'content-type': 'application/json',
   'content-length': '349',
   'connection': 'keep-alive',
   'x-amzn-requestid': '5cd594f7-175a-4e43-8a5b-d56a9d578461',
   'x-amz-apigw-id': 'aw1gZEuKoAMEkAg=',
   'x-amzn-trace-id': 'Root=1-66903602-6aababd31e78d0b417e5765a'},
  'RetryAttempts': 0},
 'agentAlias': {'agentAliasArn': 'arn:aws:bedrock:us-east-1:339712993987:agent-alias/TOCGNTDIUP/CK3K85OZRJ',
  'agentAliasId': 'CK3K85OZRJ',
  'agentAliasName': 'l279d2ba-workshop-alias',
  'agentAliasStatus': 'CREATING',
  'agentId': 'TOCGNTDIUP',
  'createdAt': datetime.datetime(2024, 7, 11, 19, 44, 2, 383218, tzinfo=tzlocal()),
  'routingConfiguration': [{}],
  'updatedAt': datetime.datetime(2024, 7, 11, 19, 44, 2, 383218, tzinfo=tzlocal())}}

### <a name="4">Invoke agent</a>
(<a href="#0">Go to top</a>)

Now that we've created the agent, let's use the `bedrock-agent-runtime` client to invoke this agent and perform some tasks.

In [21]:
USER_PROMPT_TEMPLATE = """Question: {question}

Given an input question, you will use the existing database schemas in the Knowledge Base to create a syntactically correct SQLite query and then you will use the functions and API provided to EXECUTE the SQL Query generated previously to answer the user question. 

Make sure to use only existing columns and tables based on the database schema. Make sure to wrap table names with square brackets. Make sure to add a semicolon after the end of the SQL statement generated.
Remove any backticks and any html tags like <table><th><tr> in the final response.


"""

In [22]:
question = "What are the total sales amounts by year?"

In [23]:
# Extract the agentAliasId from the response
agent_alias_id = agent_alias['agentAlias']['agentAliasId']
agent_alias_id

'CK3K85OZRJ'

In [24]:
%%time

session_id:str = str(uuid.uuid1()) # random identifier
enable_trace:bool = True
end_session:bool = False
final_answer = None

print(f"session_id :::: {session_id}")

final_answer = invoke_agent_generate_response(bedrock_agent_runtime_client,
                                               USER_PROMPT_TEMPLATE.format(question=question),
                                               agent_id, 
                                               agent_alias_id, 
                                               session_id, 
                                               enable_trace,
                                               end_session,
                                               trace_filename_prefix = 'lab2b_agent_trace',
                                               turn_number = 1)

session_id :::: fe9c74fc-3fbd-11ef-884e-16ffc4ecb111


[2024-07-11 19:45:22,962] p364 {agents_utils.py:87} INFO - Final answer ->
Year | Total Sales
2012 | $18,823,201.72  
2013 | $38,633,120.01
2014 | $38,870,148.13
2015 | $41,423,456.72
2016 | $40,568,672.36
2017 | $40,209,904.23
2018 | $38,326,623.43
2019 | $38,516,963.86
2020 | $38,862,436.79
2021 | $41,355,549.74
2022 | $39,742,066.18
2023 | $33,054,490.00


CPU times: user 56.1 ms, sys: 763 µs, total: 56.9 ms
Wall time: 50.5 s


In [25]:
# And here is the response if you just want to see agent's reply
format_final_response(question=question, final_answer=final_answer, lab_number="2b", turn_number="1", gen_sql=True)

,User Question,Agent Generated SQL,Agent Answer
0,What are the total sales amounts by year?,"SELECT strftime('%Y', [OrderDate]) AS [Year], SUM([Quantity] * [UnitPrice] * (1 - [Discount])) AS TotalSales FROM [Order Details] JOIN Orders ON [Order Details].[OrderID] = [Orders].[OrderID] GROUP BY [Year] ORDER BY [Year]","Year | Total Sales2012 | \$18,823,201.72 2013 | \$38,633,120.012014 | \$38,870,148.132015 | \$41,423,456.722016 | \$40,568,672.362017 | \$40,209,904.232018 | \$38,326,623.432019 | \$38,516,963.862020 | \$38,862,436.792021 | \$41,355,549.742022 | \$39,742,066.182023 | \$33,054,490.00"


In [26]:
# Deep dive into Agent-workflow steps in each of the tabs
%load_ext autoreload
%autoreload 2
from mlu_utils.show_trace_widget import *

show_tabs(trace_filename_prefix = 'lab2b_agent_trace', turn_number = 1)
display(out_2b_tabs_1)

trace_filename_prefix = lab2b_agent_trace and turn_number = 1


ToggleButtons(button_style='success', description='Agent Trace: (Last Logged trace only) Full trace available …

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [27]:
# Wait for tab output async processes to complete
time.sleep(10)

In [28]:
%%time
# Using Claude-v3 Haiku to generate a summary of the agent's workflow for this conversation turn
summary = summarize_agent_trace(trace_file_base_path= "trace_files/", lab_number="2b", turn_number="1")
with out_2b_summary:
    out_2b_summary.clear_output()
    display(Markdown(summary))

>>>>>>>> complete_log_path to summarize==> trace_files/full_trace_lab2b_agent_trace_1.log


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


CPU times: user 194 ms, sys: 26.9 ms, total: 221 ms
Wall time: 7.53 s


In [29]:

## Display the summary of the agent workflow trace
display(out_2b_summary)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

### <a name="9">[Be Frugal] Clean up resources </a>
(<a href="#0">Go to top</a>)


##### In the following cell, we offer the option to raise an exception to avoid auto-executing the next block of lines and optionally clean up all resources. This is useful when the `Kernel > run all` option is used.

`Please be frugal if you choose to enable this exception in the code cell below. By default it is disabled and all resources will be cleaned up immediately to avoid additional costs.`

##### Within the same kernel session, this will allow experimentation with different prompts without having to recreate agent resources (takes ~5 minutes)

In [30]:
# this avoids auto-cleanup
#raise Exception('Avoiding Auto-Cleanup of Bedrock Agent Resources')

In [31]:
%%time
cleanup_infrastructure(agent_action_group_response, lambda_name, lambda_function, lambda_role_name, agent_id, agent_alias_id, agent_role_name, bucket_name, schema_key, agent_bedrock_policy, agent_s3_schema_policy, agent_kb_schema_policy, kb_db_bedrock_policy, kb_db_aoss_policy, kb_db_s3_policy, kb_db_role_name, kb_db_collection_name, kb_db_opensearch_collection_response, knowledge_base_db_id)

bucket_name ::: l279d2ba-agent-kb-339712993987
Cleanup completed >>>>>>> 
CPU times: user 133 ms, sys: 8.47 ms, total: 141 ms
Wall time: 3.33 s


---

### <a name="10">Challenge Exercise and Lab Quiz </a>
(<a href="#0">Go to top</a>)




### Challenge Exercise 

<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <h2><i>Try it Yourself!</i></h2>
    <br>
    <p style="text-align:center;margin:auto;"><img src="./images/challenge.png" alt="Challenge" width="100" /> </p>
    <p style=" text-align: center; margin: auto;">Try the following exercise to harness the power of Bedrock agents we have taught you so far.</p>
    <br>
</div>


Try a new set of questions to run against the database.

- Get alphabetical list of products
- For each order, calculate a subtotal for each order (identified by OrderID)?
- For each employee, get their sales amount, broken down by country name

### Lab Quiz

Well done on completing the lab! Now, it's time for a brief knowledge assessment.

<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <h2><i>Try it Yourself!</i></h2>
    <br>
    <p style="text-align:center;margin:auto;"><img src="./images/activity.png" alt="Challenge" width="100" /> </p>
    <p style=" text-align: center; margin: auto;">Answer the following questions to test your understanding of Bedrock Agents.</p>
    <br>
</div>

In [32]:
from mlu_utils.agents_quiz import *

lab2b_question1

In [33]:
lab2b_question2

## Conclusion
We have now experimented with using `boto3` SDK to create, invoke and delete an agent.

### Take aways
- Adapt this notebook to create new agents for your application

## Thank You